In [1]:
%pip install numpy 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import numpy as np
from PIL import Image
import random
import math
import pickle
import os 
import matplotlib.pyplot as plt

In [4]:
#opening pkl file
import pickle 
with open('scrambled_puzzle_data.pkl', 'rb') as d:
    data_set = pickle.load(d)
    print(data_set)

[{'data': array([[[110,  50,  63],
        [ 92,  45,  66],
        [ 67,  32,  61],
        ...,
        [ 47,  55,  84],
        [ 48,  59,  89],
        [ 50,  59,  90]],

       [[106,  41,  55],
        [ 88,  36,  59],
        [ 68,  30,  61],
        ...,
        [ 47,  55,  84],
        [ 47,  58,  88],
        [ 49,  58,  87]],

       [[100,  29,  44],
        [ 84,  26,  50],
        [ 74,  31,  62],
        ...,
        [ 48,  56,  84],
        [ 46,  56,  85],
        [ 47,  55,  84]],

       ...,

       [[ 49,  45,  78],
        [ 52,  44,  75],
        [ 54,  42,  72],
        ...,
        [ 57,  52,  73],
        [ 53,  49,  73],
        [ 51,  49,  74]],

       [[ 51,  36,  66],
        [ 53,  34,  60],
        [ 56,  32,  57],
        ...,
        [ 57,  52,  72],
        [ 53,  50,  73],
        [ 51,  49,  74]],

       [[ 54,  27,  55],
        [ 56,  25,  48],
        [ 62,  25,  47],
        ...,
        [ 57,  53,  71],
        [ 53,  51,  71],
        [ 51, 

In [6]:
with open('scrambled_puzzle_data.pkl', 'rb') as f:
    dataset = pickle.load(f)
    print(dataset)

[{'data': array([[[110,  50,  63],
        [ 92,  45,  66],
        [ 67,  32,  61],
        ...,
        [ 47,  55,  84],
        [ 48,  59,  89],
        [ 50,  59,  90]],

       [[106,  41,  55],
        [ 88,  36,  59],
        [ 68,  30,  61],
        ...,
        [ 47,  55,  84],
        [ 47,  58,  88],
        [ 49,  58,  87]],

       [[100,  29,  44],
        [ 84,  26,  50],
        [ 74,  31,  62],
        ...,
        [ 48,  56,  84],
        [ 46,  56,  85],
        [ 47,  55,  84]],

       ...,

       [[ 49,  45,  78],
        [ 52,  44,  75],
        [ 54,  42,  72],
        ...,
        [ 57,  52,  73],
        [ 53,  49,  73],
        [ 51,  49,  74]],

       [[ 51,  36,  66],
        [ 53,  34,  60],
        [ 56,  32,  57],
        ...,
        [ 57,  52,  72],
        [ 53,  50,  73],
        [ 51,  49,  74]],

       [[ 54,  27,  55],
        [ 56,  25,  48],
        [ 62,  25,  47],
        ...,
        [ 57,  53,  71],
        [ 53,  51,  71],
        [ 51, 

In [7]:
output_dir = 'processe_images'
os.makedirs(output_dir, exist_ok=True)

In [11]:
for idx, item in enumerate(dataset):
    image_array = item['data']
    label = item.get('correct_index', 'unknown')
    rotation = item.get('applied_rotation_deg', 0)

In [13]:
image_array = np.array(image_array).astype(np.uint8)
image_pil = Image.fromarray(image_array)
if rotation != 0:
        image_pil = image_pil.rotate(-rotation, expand=True)
gray_image = image_pil.convert('L')
resized_image = gray_image.resize((128, 128))
filename = f'image_{idx:03d}_label_{label}.png'
filepath = os.path.join(output_dir, filename)
resized_image.save(filepath)
print(f"Saved {filename}")
print("All images processed and saved.")



Saved image_080_label_42.png
All images processed and saved.


In [15]:
with open("scrambled_puzzle_data.pkl", "rb") as f:
    scrambled_data = pickle.load(f)

print(f"Loaded {len(scrambled_data)} pieces.")

Loaded 81 pieces.


In [17]:
def rotate_piece(image_array, angle):
    return np.array(Image.fromarray(image_array).rotate(angle, expand=False))


In [18]:
def get_edges(image):
    return {
        'top': image[0, :, :],
        'bottom': image[-1, :, :],
        'left': image[:, 0, :],
        'right': image[:, -1, :]
    }


In [19]:
def edge_similarity(edge1, edge2):
    return np.mean((edge1.astype(np.float32) - edge2.astype(np.float32)) ** 2)


In [20]:
def get_all_rotations(piece):
    rotations = {}
    for angle in [0, 90, 180, 270]:
        rotated = rotate_piece(piece, angle)
        rotations[angle] = rotated
    return rotations


In [21]:
def find_best_matches(pieces):
    matches = {}
    for i, piece in enumerate(pieces):
        best_right = (None, None, float('inf'))  # (match_idx, rotation, score)
        best_bottom = (None, None, float('inf'))

        for j, other in enumerate(pieces):
            if i == j:
                continue
            for angle in [0, 90, 180, 270]:
                rotated = rotate_piece(other, angle)
                edges_other = get_edges(rotated)
                edges_piece = get_edges(piece)

                # Compare right edge of piece to left edge of other
                score_right = edge_similarity(edges_piece['right'], edges_other['left'])
                if score_right < best_right[2]:
                    best_right = (j, angle, score_right)

                # Compare bottom edge of piece to top edge of other
                score_bottom = edge_similarity(edges_piece['bottom'], edges_other['top'])
                if score_bottom < best_bottom[2]:
                    best_bottom = (j, angle, score_bottom)

        matches[i] = {
            'right': best_right,
            'bottom': best_bottom
        }
    return matches


In [22]:
def assemble_puzzle(pieces, matches):
    grid = [[None for _ in range(9)] for _ in range(9)]
    used = set()

    # Start with an arbitrary piece
    current_idx = 0
    current_angle = 0
    used.add(current_idx)
    grid[0][0] = rotate_piece(pieces[current_idx], current_angle)

    for row in range(9):
        for col in range(1, 9):
            prev_idx = current_idx
            match_idx, angle, _ = matches[prev_idx]['right']
            while match_idx in used:
                match_idx, angle, _ = matches[match_idx]['right']
            used.add(match_idx)
            current_idx = match_idx
            grid[row][col] = rotate_piece(pieces[match_idx], angle)

        if row < 8:
            # Move to the start of the next row
            first_in_row_idx, angle, _ = matches[current_idx]['bottom']
            while first_in_row_idx in used:
                first_in_row_idx, angle, _ = matches[first_in_row_idx]['bottom']
            used.add(first_in_row_idx)
            current_idx = first_in_row_idx
            grid[row + 1][0] = rotate_piece(pieces[first_in_row_idx], angle)

    return grid


In [23]:
def stitch_grid(grid):
    rows = [np.hstack(row) for row in grid]
    full_image = np.vstack(rows)
    return full_image


In [28]:
# Extract raw image arrays
pieces = [item['data'] for item in scrambled_data]

# Find best matches
matches = find_best_matches(pieces)

# Assemble the puzzle
grid = assemble_puzzle(pieces, matches)

# Stitch and save
final_image = stitch_grid(grid)
Image.fromarray(final_image.astype(np.uint8)).save("solved_puzzle.png")
print("✅ Puzzle solved and saved as 'solved_puzzle.png'")



AxisError: axis 2 is out of bounds for array of dimension 2

In [26]:

!pip install tqdm
from tqdm import tqdm
import numpy as np

def to_grayscale(edge):
    return np.mean(edge, axis=2)  # Convert RGB to grayscale

def get_edges(image):
    return {
        'top': to_grayscale(image[0, :, :]),
        'bottom': to_grayscale(image[-1, :, :]),
        'left': to_grayscale(image[:, 0, :]),
        'right': to_grayscale(image[:, -1, :])
    }

def edge_similarity(edge1, edge2):
    return np.mean((edge1 - edge2) ** 2)

def find_best_matches(pieces):
    # Precompute all rotations and edges
    rotated_pieces = [
        {angle: rotate_piece(piece, angle) for angle in [0, 90, 180, 270]}
        for piece in pieces
    ]

    edge_cache = [
        {angle: get_edges(rotated_pieces[i][angle]) for angle in [0, 90, 180, 270]}
        for i in range(len(pieces))
    ]

    matches = {}

    for i in tqdm(range(len(pieces)), desc="Matching pieces"):
        best_right = (None, None, float('inf'))
        best_bottom = (None, None, float('inf'))

        for j in range(len(pieces)):
            if i == j:
                continue

            for angle in [0, 90, 180, 270]:
                edge_i_right = edge_cache[i][0]['right']  # Always compare original orientation
                edge_i_bottom = edge_cache[i][0]['bottom']

                edge_j_left = edge_cache[j][angle]['left']
                edge_j_top = edge_cache[j][angle]['top']

                score_right = edge_similarity(edge_i_right, edge_j_left)
                score_bottom = edge_similarity(edge_i_bottom, edge_j_top)

                if score_right < best_right[2]:
                    best_right = (j, angle, score_right)

                if score_bottom < best_bottom[2]:
                    best_bottom = (j, angle, score_bottom)

        matches[i] = {
            'right': best_right,
            'bottom': best_bottom
        }

    return matches



Defaulting to user installation because normal site-packages is not writeable


In [32]:
from tqdm import tqdm
import numpy as np

def to_grayscale(edge):
    print("Edge shape:", edge.shape)
    if edge.ndim == 3:
       return np.mean(edge, axis=2)  # Convert RGB to grayscale
    return edge

def get_edges(image):
    return {
        'top': to_grayscale(image[0, :, :]),
        'bottom': to_grayscale(image[-1, :, :]),
        'left': to_grayscale(image[:, 0, :]),
        'right': to_grayscale(image[:, -1, :])
    }

def edge_similarity(edge1, edge2):
    return np.mean((edge1 - edge2) ** 2)

def find_best_matches(pieces):
    # Precompute all rotations and edges
    rotated_pieces = [
        {angle: rotate_piece(piece, angle) for angle in [0, 90, 180, 270]}
        for piece in pieces
    ]

    edge_cache = [
        {angle: get_edges(rotated_pieces[i][angle]) for angle in [0, 90, 180, 270]}
        for i in range(len(pieces))
    ]

    matches = {}

    for i in tqdm(range(len(pieces)), desc="Matching pieces"):
        best_right = (None, None, float('inf'))
        best_bottom = (None, None, float('inf'))

        for j in range(len(pieces)):
            if i == j:
                continue

            for angle in [0, 90, 180, 270]:
                edge_i_right = edge_cache[i][0]['right']  # Always compare original orientation
                edge_i_bottom = edge_cache[i][0]['bottom']

                edge_j_left = edge_cache[j][angle]['left']
                edge_j_top = edge_cache[j][angle]['top']

                score_right = edge_similarity(edge_i_right, edge_j_left)
                score_bottom = edge_similarity(edge_i_bottom, edge_j_top)

                if score_right < best_right[2]:
                    best_right = (j, angle, score_right)

                if score_bottom < best_bottom[2]:
                    best_bottom = (j, angle, score_bottom)

        matches[i] = {
            'right': best_right,
            'bottom': best_bottom
        }

    return matches


In [33]:
def assemble_puzzle(pieces, matches):
    grid = [[None for _ in range(9)] for _ in range(9)]
    used = set()

    # Start with piece 0
    current_idx = 0
    current_angle = 0
    used.add(current_idx)
    grid[0][0] = rotate_piece(pieces[current_idx], current_angle)

    for row in range(9):
        for col in range(1, 9):
            prev_idx = current_idx
            match_idx, angle, _ = matches[prev_idx]['right']
            while match_idx in used:
                match_idx, angle, _ = matches[match_idx]['right']
            used.add(match_idx)
            current_idx = match_idx
            grid[row][col] = rotate_piece(pieces[match_idx], angle)

        if row < 8:
            first_in_row_idx, angle, _ = matches[current_idx]['bottom']
            while first_in_row_idx in used:
                first_in_row_idx, angle, _ = matches[first_in_row_idx]['bottom']
            used.add(first_in_row_idx)
            current_idx = first_in_row_idx
            grid[row + 1][0] = rotate_piece(pieces[first_in_row_idx], angle)

    return grid


In [34]:
def stitch_grid(grid):
    rows = [np.hstack(row) for row in grid]
    full_image = np.vstack(rows)
    return full_image


In [ ]:
from PIL import Image

# Extract raw image arrays
pieces = [item['data'] for item in scrambled_data]

# Run your optimized matching
matches = find_best_matches(pieces)

# Assemble the puzzle
grid = assemble_puzzle(pieces, matches)

# Stitch and save
final_image = stitch_grid(grid)
Image.fromarray(final_image.astype(np.uint8)).save("solved_puzzle.png")
print(" Puzzle solved and saved as 'solved_puzzle.png'")


Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)
Edge shape: (50, 3)


Matching pieces: 100%|██████████| 81/81 [00:00<00:00, 183.65it/s]
